<a href="https://colab.research.google.com/github/avats-dev/ML4CV/blob/main/ML4CV_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Deep learning based on transfer learning for plant disease detection

VGG19 is used as base model and inception block (derived from googlenet) is used over base for this model and then a global average pooling (GAP) is used followed by softmax classifier.
VGG19 is trained on imagenet dataset.

This model would be trained on head layers with plant village dataset.

INC-VGGN Architecture.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
#!ls

In [ ]:
#!unzip dataset.zip

In [7]:
%cd dataset

/content/drive/My Drive/dataset


In [9]:
!ls -l

total 12
drwx------ 5 root root 4096 Feb  7 14:44 maize
drwx------ 5 root root 4096 Feb  7 14:44 maize_plantvillage
drwx------ 5 root root 4096 Feb  7 14:44 rice


In [10]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1/255)

In [13]:
train_data = datagen.flow_from_directory('rice/train', target_size=(224,224),
                            batch_size=10, class_mode='categorical')
test_data = datagen.flow_from_directory('rice/test', target_size=(224,224),
                            batch_size=10, class_mode='categorical',
                            shuffle=True, seed=33)
val_data = datagen.flow_from_directory('rice/val', target_size=(224,224),
                            batch_size=10, class_mode='categorical',
                            shuffle=True, seed=33)


Found 350 images belonging to 5 classes.
Found 60 images belonging to 5 classes.
Found 150 images belonging to 5 classes.


In [14]:
import tensorflow as tf
base = tf.keras.applications.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
)

80142336/80134624 [==============================] - 0s 0us/step


In [15]:
base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
for layer in base.layers:
  layer.trainable = False

base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras import backend as k

def swish(x):
  return (k.sigmoid(x)*x)

get_custom_objects().update({'swish': Activation(swish)})

In [25]:
from keras.layers import Conv2D, BatchNormalization, Lambda

def conv_bn(x, nb_filters, kernel_size, padding, strides):
  x = Conv2D(nb_filters, kernel_size, padding=padding,
             strides=strides)(x)
  x = Lambda(swish)(x)
  x = BatchNormalization(axis=3)(x)
  return x

In [30]:
from keras.layers import MaxPooling2D, concatenate

def inception(x, nb_filters):
  branch1 = conv_bn(x, nb_filters, (1,1), padding='same',
                    strides=(1,1))
  branch1_2 = conv_bn(branch1, nb_filters, (3,3), padding='same',
                    strides=(1,1))
  branch1_3 = conv_bn(branch1_2, nb_filters, (3,3), padding='same',
                    strides=(1,1))
  
  branch2 = conv_bn(x, nb_filters, (1,1), padding='same',
                    strides=(1,1))
  branch2_1 = conv_bn(branch2, nb_filters, (3,3), padding='same',
                    strides=(1,1))
  
  branch3 = MaxPooling2D(pool_size=(3,3), padding='same', strides=(1,1))(x)
  branch3_1 = conv_bn(branch3, nb_filters, (1,1), padding='same',
                    strides=(1,1))
  
  branch4 = conv_bn(x, nb_filters, (1,1), padding='same',
                    strides=(1,1))
  
  res = concatenate([branch1_2, branch2_1, branch3_1, branch4],
              axis=3)

  return res
  
  

In [31]:
from keras.layers import GlobalAveragePooling2D, Dense

base_out = base.layers[-6].output
base_out = conv_bn(base_out, 512, (3,3), padding='same',
                    strides=(1,1))
base_out = MaxPooling2D(pool_size=(2,2), strides=(1,1),
                        padding='same')(base_out)

out = inception(base_out, 512)
out = inception(out, 512)

out = GlobalAveragePooling2D()(out)

output = Dense(5, activation='softmax')(out)


from keras.models import Model
vggmodel = Model(inputs = base.input, outputs=output)

vggmodel.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [32]:
from keras.optimizers import SGD

learning_rate = 0.0001
decay = 1e-6
momentum = 0.8
nest = True

sgd = SGD(lr=learning_rate, decay=decay, momentum=momentum,
    nesterov=nest)

vggmodel.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy']
)



In [33]:
history = vggmodel.fit(train_data, epochs=30, 
                       validation_data=val_data, verbose=1)

Epoch 1/30
35/35 [==============================] - 15s 162ms/step - loss: 1.5594 - accuracy: 0.3262 - val_loss: 1.6859 - val_accuracy: 0.4533
Epoch 2/30
35/35 [==============================] - 5s 152ms/step - loss: 1.1866 - accuracy: 0.6160 - val_loss: 1.4210 - val_accuracy: 0.5600
Epoch 3/30
35/35 [==============================] - 5s 153ms/step - loss: 0.9858 - accuracy: 0.7103 - val_loss: 1.2216 - val_accuracy: 0.5867
Epoch 4/30
35/35 [==============================] - 5s 155ms/step - loss: 0.9030 - accuracy: 0.7074 - val_loss: 1.0570 - val_accuracy: 0.6400
Epoch 5/30
35/35 [==============================] - 5s 157ms/step - loss: 0.7640 - accuracy: 0.8168 - val_loss: 0.9227 - val_accuracy: 0.7467
Epoch 6/30
35/35 [==============================] - 6s 158ms/step - loss: 0.6983 - accuracy: 0.8053 - val_loss: 0.8164 - val_accuracy: 0.7933
Epoch 7/30
35/35 [==============================] - 6s 158ms/step - loss: 0.6455 - accuracy: 0.8769 - val_loss: 0.7320 - val_accuracy: 0.7933
Epoch

In [34]:
vggmodel.evaluate(test_data, verbose=1)

6/6 [==============================] - 1s 88ms/step - loss: 0.3323 - accuracy: 0.9000


[0.3323075473308563, 0.8999999761581421]